In [85]:
import numpy as np
import pandas as pd
import folium
import json
import mimetypes
import http.client

In [86]:
df_g = pd.read_excel('Datasets/wiidcountry_0.xlsx')

In [87]:
sel_columns = ['id' , 'country' , 'c3', 'year' , 	'region_wb',  'incomegroup'	, 'population', 'gdp', 'gini', 'palma', 'middle50', 'bottom5', 'top5']

df_g1 = df_g[sel_columns]

In [88]:
df_g1.shape
df_g1 = df_g1.replace(['United States'],'United States of America')


In [89]:
df1 = df_g1.groupby('country', as_index=False).last()


In [90]:
df2 = df1.drop(['id', 'c3', 'region_wb'], axis =1)
df2.head()
df2.columns

Index(['country', 'year', 'incomegroup', 'population', 'gdp', 'gini', 'palma',
       'middle50', 'bottom5', 'top5'],
      dtype='object')

In [91]:
# Generating the base map
income_map = folium.Map(tiles='OpenStreetMap', min_zoom = 1.5)

In [92]:
income_map

In [93]:
#Obtaining the Geodata. 
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

# The Above URL contains the coordinates of each countries boundaries (Polygons to be drawn on the map!)

In [94]:
# Generating Choropleth Map Layer
folium.Choropleth(
    geo_data = country_shapes,
    min_zoom = 2,
    name = 'Income Inequality Dashboard',
    data = df2,
    columns = ['country','gini', 'year', 'incomegroup'],
    key_on = 'feature.properties.name',
    fill_color = 'YlOrRd',
    nan_fill_color='grey',
    nan_fill_opacity= '.2',
    legend_name='Income Inequality Metrics',
).add_to(income_map)
income_map

In [95]:
# Now, we'll generate markers for each country:
#Appending text to the columns of the data frame, for those that will be shownn on the map! 
df2.update(df2['population'].map("Latest Population: {}".format))
df2.update(df2['gdp'].map('GDP: {}'.format))
df2.update(df2['gini'].map("Gini Index: {}".format))
df2.update(df2['palma'].map("Palma Ratio: {}".format))
df2.update(df2['top5'].map("Top 5%'s share': {}".format))

In [112]:
xy_url ='https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv'
xy = pd.read_csv(xy_url)
xy.rename(columns={"Country":"country"}, inplace = True)


In [99]:
df_merge = pd.merge(df2, xy, on = 'country')

In [100]:
#Writing a function for plotting the markers on each country, with a popup!
def map_plot(xy):
    folium.CircleMarker(location = [xy.latitude, xy.longitude],
                        radius = 2, 
                        weight = 3,
                        popup = [xy.country,
                                 xy.population, 
                                 xy.gdp, 
                                 xy.gini, 
                                 xy.palma, 
                                 xy.top5], 
                        fill_color = "#3186cc", 
                        fill = True).add_to(income_map)

In [101]:
df_merge.apply(map_plot, axis=1 )
income_map.fit_bounds(income_map.get_bounds())
income_map